In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def ipu(datafile, constrains):
    df = pd.read_csv(datafile)
    ipf = constrains

    e = np.finfo('float').eps
    for i in ipf:
        if i == 0:
            i = e
    # w = np.array(df['weights'], dtype=np.float32)
    w=np.ones(len(df))
    # df = df.drop(['ID'], axis=1)
    # df = df.drop(['weights'], axis=1)
    ws = np.zeros(len(df))
    i = 0

    for col  in df.columns:
        x = 0
        for j in range(len(df)) :
            print(w[j], df[col][j])
            x += w[j]*df[col][j]
        ws[i] = x
        i += 1
    print('weighted sums = ', ws)

    delta = sum((abs(ws-ipf))/ipf)
    print(delta)

    a = df.to_numpy(dtype=np.float32)
    # print(a.shape)
    # print(df.shape)
    print(a)
    print(type(a[1][1]))

    w1 = open("ipu-weights.txt", "w")
    w2 = open("ipu-weighted_sums.txt", "w")
    
    t=1

    while(True):

        delta_prev = delta

        w1.write('iteration'+str(t)+'\n')
        w2.write('iteration'+str(t)+'\n')

        for i in range(df.shape[1]):
            pws = ws[i]
            ws[i] = ipf[i]
            multiplier = ws[i]/pws
            # print(multiplier)
        
            for j in range(df.shape[0]):
                if(a[j][i] == 0):
                    continue
                w[j] *= multiplier
        
            for k in range(df.shape[1]):
                x = 0
                for j in range(df.shape[0]):
                    x += a[j][k]*w[j]
                ws[k] = x
        
            w1.write('weights'+str(i)+str(w)+'\n')
            w2.write("weighted sum"+str(i+1)+str(ws)+'\n')

        delta = sum((abs(ws-ipf))/ipf)
        if(abs(delta_prev-delta) < .0001):
            break
        t+=1




In [24]:
f1=open('person_constrains.txt','r')
f1=f1.read().split()
person=list(f1)
f1=open('hhold_constrains.txt','r')
f1=f1.read().split()
hhold=list(f1)
ipf=np.array(hhold+person)
x = ipu('binary_matrix.csv', ipf)
ipf

1.0 2017000000496
1.0 2017000000567
1.0 2017000000954
1.0 2017000001569
1.0 2017000001843
1.0 2017000002261
1.0 2017000002716
1.0 2017000002867
1.0 2017000003161
1.0 2017000003442
1.0 2017000003685
1.0 2017000003932
1.0 2017000005310
1.0 2017000005678
1.0 2017000005726
1.0 2017000006164
1.0 2017000006318
1.0 2017000006765
1.0 2017000007407
1.0 2017000007825
1.0 2017000008274
1.0 2017000008557
1.0 2017000008700
1.0 2017000008742
1.0 2017000009471
1.0 2017000009492
1.0 2017000009705
1.0 2017000010614
1.0 2017000011181
1.0 2017000011676
1.0 2017000011795
1.0 2017000011889
1.0 2017000012735
1.0 2017000013076
1.0 2017000013147
1.0 2017000013654
1.0 2017000014114
1.0 2017000014315
1.0 2017000014674
1.0 2017000015581
1.0 2017000016092
1.0 2017000016185
1.0 2017000017688
1.0 2017000017938
1.0 2017000018560
1.0 2017000018599
1.0 2017000018745
1.0 2017000019932
1.0 2017000020709
1.0 2017000020938
1.0 2017000021233
1.0 2017000021691
1.0 2017000021778
1.0 2017000023137
1.0 2017000023806
1.0 201700

C:\Users\abubabu\AppData\Local\Temp\ipykernel_25344\2164168956.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(datafile)


TypeError: can't multiply sequence by non-int of type 'numpy.float64'

In [ ]:
xx=pd.read_csv('binary_matrix.csv')
# print(xx.columns)
xx.rename(columns={xx.columns[0]: 'SERIALNO'}, inplace=True)
for idx, row in xx.iterrows():
    if row['SERIALNO']=='2017001221812':
        print(row.dtypes)
        break


C:\Users\abubabu\AppData\Local\Temp\ipykernel_25344\2025125891.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  xx=pd.read_csv('binary_matrix.csv')


object
